## IRS

   - An IRS is a financial derivative contract in which two parties exchange cash flows of interest payments.
   - Typically, one party pays a fixed rate (fixed leg), while the other party pays a floating rate (floating leg).
   - The value of an IRS can be determined using the present value of the fixed and floating legs.

## Caplets and Floorlets

   - Caplets and floorlets are components of interest rate caps and floors, respectively. A caplet is a financial derivative that pays off when the interest rate exceeds a predetermined strike rate, whereas a floorlet pays off when the interest rate falls below a predetermined strike rate.
   - The formulas for caplets and floorlets involve several variables:
     - $ P(t, T_{i-1}) $: Discount factor from time $ t $ to $ T_{i-1} $
     - $ K $: Strike rate
     - $ Z_{T_i, T_{i-1}}^t $: Forward rate observed at time $ t $ for the period from $ T_{i-1} $ to $ T_i $
     - $ v(t) $: Volatility term
     - $ \Phi $: Cumulative distribution function (CDF) of the standard normal distribution
     - $ d_1 $ and $ d_2 $ are auxiliary variables defined as:
       
       $d_1(K, F, v) = \frac{\log(F/K) + \frac{v^2}{2}}{v}$
       
       
       $d_2(K, F, v) = d_1(K, F, v) - v$

$Caplet_i(t) = \frac{P(t,T_{i-1})}{K^{*}}\left[K\Phi\left(-d_2(K,Z_{T_i,T_{i-1}}^t ,v(t))\right)-Z_{T_i,T_{i-1}}^t  \Phi \left(-d_1(K, Z_{T_i,T_{i-1}}^t, v(t))\right)\right]$

$Floorlet_i(t) = \frac{P(t,T_{i-1})}{K^{*}}\left[-K\Phi\left(-d_2(K,Z_{T_i,T_{i-1}}^t ,v(t))\right)+Z_{T_i,T_{i-1}}^t  \Phi \left(-d_1(K, Z_{T_i,T_{i-1}}^t, v(t))\right)\right]$

where $Z_{S,T}^t=\frac{P(t,S)}{P(t,T)}$, $d_1(K,F,v)=\frac{(log(\frac{F}{K})+\frac{v^2}2)}{v}$, $d_2(K,F,v)=d_1(K,F,v)-v$

## Implementation Steps

   - **Step 1: Calculate the discount factor \( P(t, T_{i-1}) \) using the Hull-White model.**
   - **Step 2: Calculate the forward rate \( Z_{T_i, T_{i-1}}^t \).**
   - **Step 3: Determine the volatility term \( v(t) \).**
   - **Step 4: Compute the \( d_1 \) and \( d_2 \) variables.**
   - **Step 5: Calculate the caplet and floorlet prices using the provided formulas.**

In [11]:
from scipy.stats import norm
import numpy as np

In [12]:
def discount_factor(t, T, r, alpha, sigma):
    B = (1 - np.exp(-alpha * (T - t))) / alpha
    A = np.exp((B - (T - t)) * (theta - 0.5 * sigma ** 2 / alpha ** 2) - (sigma ** 2 * B ** 2) / (4 * alpha))
    return A * np.exp(-B * r)

In [13]:
def forward_rate(P_t_T1, P_t_T2, T1, T2):
    return (P_t_T1 / P_t_T2 - 1) / (T2 - T1)

In [14]:
def volatility(t, T1, T2, sigma):
    return sigma * np.sqrt(T2 - T1)

In [15]:
def d1_d2(K, F, v):
    d1 = (np.log(F / K) + 0.5 * v ** 2) / v
    d2 = d1 - v
    return d1, d2

In [16]:
def K_star(alpha_i, K):
    return 1 / (1 + alpha_i * K)

In [17]:
def caplet_price(P_t_T1, K, Z_Ti_T1, v, alpha_i):
    K_star_val = K_star(alpha_i, K)
    d1, d2 = d1_d2(K, Z_Ti_T1, v)
    caplet = P_t_T1 / K_star_val * (K * norm.cdf(-d2) - Z_Ti_T1 * norm.cdf(-d1))
    return caplet

In [18]:
def floorlet_price(P_t_T1, K, Z_Ti_T1, v, alpha_i):
    K_star_val = K_star(alpha_i, K)
    d1, d2 = d1_d2(K, Z_Ti_T1, v)
    floorlet = P_t_T1 / K_star_val * (-K * norm.cdf(-d2) + Z_Ti_T1 * norm.cdf(-d1))
    return floorlet

### Example

In [19]:
# Example parameters
t = 0
T1 = 1
T2 = 2
r = 0.03
alpha = 0.1
sigma = 0.02
theta = 0.03
K = 0.025
alpha_i = 0.5  # Example day count fraction for the period

In [20]:
# Calculate discount factors
P_t_T1 = discount_factor(t, T1, r, alpha, sigma)
P_t_T2 = discount_factor(t, T2, r, alpha, sigma)

In [21]:
# Calculate forward rate
Z_Ti_T1 = forward_rate(P_t_T1, P_t_T2, T1, T2)

In [22]:
# Calculate volatility
v = volatility(t, T1, T2, sigma)

In [23]:
# Calculate caplet and floorlet prices
caplet = caplet_price(P_t_T1, K, Z_Ti_T1, v, alpha_i)
floorlet = floorlet_price(P_t_T1, K, Z_Ti_T1, v, alpha_i)

In [24]:
caplet, floorlet

(1.1307942868529538e-24, -1.1307942868529538e-24)

Explanation
 1. **Discount Factor**: Calculated using a simplified version of the Hull-White model.
 2. **Forward Rate**: Derived from the ratio of discount factors.
 3. **Volatility**: Simplified to be constant for demonstration purposes.
 4. **d1 and d2**: Computed using the given formulas.
 5. **K^{*}**: Defined as $\frac1{1+ \alpha K}$
 6. **Caplet and Floorlet Prices**: Derived using the provided formulas and the normal CDF.

 This implementation assumes simplified calculations for discount factors and volatility. In a real-world scenario, more complex methods and market data would be used. Adjustments might be necessary to fully align with the Hull-White model parameters and market conditions.